In [ ]:
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter
class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass

session = CachedLimiterSession(
    limiter=Limiter(RequestRate(2, Duration.SECOND*5)),  # max 2 requests per 5 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
)
session.headers['User-agent'] = 'n.finance/1.0'

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf

stocks = ['2330.TW', '2882.TW']
tickers = yf.Tickers(stocks, session=session)

In [ ]:
tickers.history(start='2015-01-01', end='2016-12-31', auto_adjust=False)

In [ ]:
df = pd.DataFrame()

for stock_code in stocks:
    price = tickers.tickers[stock_code].history(start='2015-01-01', end='2016-12-31', auto_adjust=False)
    simpleret = price['Close'].pct_change().dropna()
    simpleret.index = simpleret.index.date  # type: ignore
    simpleret.name = stock_code

    df = pd.concat([df, simpleret], axis=1)

In [ ]:
df

In [ ]:
df.describe()

## 投資組合的報酬

- 由一種以上的證券或資產所構成的集合，稱為投資組合。
- 投資組合的預期報酬率
    - 為所有個別資產預期報酬率的加權平均數。

$
\begin{aligned}
R_P &= 總收益/初始投入資本 \\
    &= \frac{(W_0 \omega_A R_A)+(W_0 \omega_B R_B)}{W_0} \\
    &= \omega_A R_A + \omega_B R_B \\
    &= \sum{\omega R}
\end{aligned}
$

$\omega_i$ 表示各資產的權重，$R_i$ 表示投資各資產所能獲得的預期報酬率。

In [ ]:
df["2330.TW"]

In [ ]:
# 以 50% 資金投資台積電，以 50% 資金投資
# 國泰金，試著算出投資組合的報酬。

weight = 0.5
portfolio_ret = weight * df["2330.TW"].mean() + (1 - weight) * df["2882.TW"].mean()
portfolio_ret

## 投資組合的風險

- 以標準差或變異數衡量投資組合的風險。

$
\begin{aligned}
&Var(W_1 R_1 + W_2 R_2) \\
= &W_1^2 Var(R_1) + W_2^2 Var(R_2) + 2 W_1 W_2 Cov(R_1, R_2) \\
= &W_1^2 \sigma_1^2 + W_2^2 \sigma_2^2 + 2 W_1 W_2 \sigma_1 \sigma_2 \rho_{1,2}
\end{aligned}
$

- 由多種資產構成的投資組合不但包含原先個別資產的風險（有權數的調整），尚隱 含個別資產間相互影響所帶來的風險。

In [ ]:
# 以 50% 資金投資台積電，以 50% 資金投資
# 國泰金，試著算出投資組合的風險。

import math


weight = 0.5
twse, cfl = df["2330.TW"], df["2882.TW"]

portfolio_var = (
    (weight**2) * (twse.std()**2) + ((1-weight)**2) *
                  (cfl.std()**2) + 2 * weight * (1-weight) *
                  twse.corr(cfl, method="pearson") * twse.std() * cfl.std()
)
math.sqrt(portfolio_var)